In [1]:
from model.data import *
from model.model import *
d = Data()

Using TensorFlow backend.


In [ ]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 10
config["kfold"] = 5

config["input_shape"] = (1, 64, 64, 64)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

# model.summary()
counter = 1
for layer in model.layers:
    print(counter, "&", layer.name, "&", layer.output_shape)
    counter += 1


In [4]:
# 2 & conv3d_1 & (None, 32, 64, 64, 64)
# 3 & activation_1 & (None, 32, 64, 64, 64)
# 4 & conv3d_2 & (None, 32, 64, 64, 64)
# 5 & activation_2 & (None, 32, 64, 64, 64)
# 6 & max_pooling3d_1 & (None, 32, 32, 32, 32)
counter = 1
filt = 32
lay = 64
while (filt < 512):
#     print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    lay = int(lay / 2)
    print("Contract " + str(counter), "&", "Max pooling & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt *= 2
    counter += 1    

    
#     2 & Conv3D + ReLU & (64, 64, 64, 64) \\
#     2 & Max pooling & (64, 32, 32, 32) \\

Contract 1 & Conv3D + ReLU & (32, 64, 64, 64) \\
Contract 1 & Max pooling & (32, 32, 32, 32) \\
Contract 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Contract 2 & Max pooling & (64, 16, 16, 16) \\
Contract 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Contract 3 & Max pooling & (128, 8, 8, 8) \\
Contract 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Contract 4 & Max pooling & (256, 4, 4, 4) \\


In [5]:
lay = 8
counter = 4
filt = 256
while (counter >= 1):
    print("Expand " + str(counter), "&", "Deconvolution3D & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Concatenation & (" + str(filt * 2) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt = int(filt/2)
    lay *= 2
    counter -= 1

Expand 4 & Deconvolution3D & (256, 8, 8, 8) \\
Expand 4 & Concatenation & (512, 8, 8, 8) \\
Expand 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Expand 3 & Deconvolution3D & (128, 16, 16, 16) \\
Expand 3 & Concatenation & (256, 16, 16, 16) \\
Expand 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Expand 2 & Deconvolution3D & (64, 32, 32, 32) \\
Expand 2 & Concatenation & (128, 32, 32, 32) \\
Expand 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Expand 1 & Deconvolution3D & (32, 64, 64, 64) \\
Expand 1 & Concatenation & (64, 64, 64, 64) \\
Expand 1 & Conv3D + ReLU & (32, 64, 64, 64) \\


In [2]:
file = [0,0.5829777736985029,0.4170222262883388,0.5996851144842315,0.400314885493062,
1,0.8044424207247803,0.19555757927892792,0.6421565902928003,0.35784340928790076,
2,0.8708705066167651,0.12912949338861276,0.6454261354491866,0.35457386387791034,
3,0.900348717422359,0.09965128258347583,0.6615997318296588,0.3384002675054874,
4,0.9187994508911332,0.08120054911129213,0.6561025248613104,0.3438974755415738,
5,0.9301866617832381,0.06981333821409062,0.6407387799607699,0.35926121985314835,
6,0.9387168464562851,0.06128315354476948,0.6566260696287656,0.3433739302886857,
7,0.9453217551403003,0.0546782448604379,0.6570898626777689,0.34291013755840094,
8,0.9503667144051974,0.04963328559311548,0.6355091406934802,0.36449085936346204,
9,0.9544701752092806,0.0455298247895595,0.6473466263245875,0.35265337336574487]

print(len(file))
for i in range(int(len(file)/5)):
    print(str(file[i * 5] + 1) + ' & ', end="")
    for j in range(1, 5):
        if j != 4:
            print("{0:.4f}".format(file[i*5 + j]) + ' & ', end="")
        else:
            print("{0:.4f}".format(file[i*5 + j]) + ' \\\\')
                

50
1 & 0.5830 & 0.4170 & 0.5997 & 0.4003 \\
2 & 0.8044 & 0.1956 & 0.6422 & 0.3578 \\
3 & 0.8709 & 0.1291 & 0.6454 & 0.3546 \\
4 & 0.9003 & 0.0997 & 0.6616 & 0.3384 \\
5 & 0.9188 & 0.0812 & 0.6561 & 0.3439 \\
6 & 0.9302 & 0.0698 & 0.6407 & 0.3593 \\
7 & 0.9387 & 0.0613 & 0.6566 & 0.3434 \\
8 & 0.9453 & 0.0547 & 0.6571 & 0.3429 \\
9 & 0.9504 & 0.0496 & 0.6355 & 0.3645 \\
10 & 0.9545 & 0.0455 & 0.6473 & 0.3527 \\
